Procedimiento:

Entrar al directorio con el archivo .sh
```txt
cd /home/airman/Escritorio/Repos_HENRY/DTS_01_Bootcamp/.DS-M4-Cluster_Hadoop

```
Verificar directorio
```
dir (muestra los archivos en el directorio)
dir -ll (muestra los archivos en el directorio, sus metadatos y permisos)

```
output

```
total 12
drwxrwxr-x 2 airman airman 4096 may 15 17:00 config
-rw-rw-r-- 1 airman airman 1700 may 15 17:00 README.md
-rw-rw-r-- 1 airman airman  877 may 15 17:00 start-container.sh

```

para poder ejercutar el archivo start-container es necesario conferirle permisos de ejecucion

ejecutamos
```
chmod 777 start-container.sh
```

output
```
total 12
drwxrwxr-x 2 airman airman 4096 may 15 17:00 config
-rw-rw-r-- 1 airman airman 1700 may 15 17:00 README.md
-rwxrwxrwx 1 airman airman  877 may 15 17:00 start-container.sh

```
1. Crear red para el cluster de hadoop

sudo docker network create --driver=bridge hadoop

2. Inicializar el cluster

sudo ./start-container.sh

output:

start hadoop-master container...
start hadoop-slave1 container...
start hadoop-slave2 container...
root@hadoop-master:~# 

dir (muestra los archivos en dfs)

output
```
root@hadoop-master:~# dir
hdfs  run-wordcount.sh	start-hadoop.sh

```

3. Iniciar hadoop
./start-hadoop.sh

Verificar directorio

```
dir -ll (muestra los archivos en el directorio, sus metadatos y permisos)
```
output:
drwxr-xr-x 1 root root 4096 Jun  5  2020 hdfs
-rwxr-xr-x 1 root root  695 Jun  1  2020 run-wordcount.sh
-rwxr-xr-x 1 root root  120 Jun  1  2020 start-hadoop.sh (ya cuenta con permisos de ejecución )

-- Revisar el localhost:8088 para ir al cluster hadoop

4. Un archivo txt de un libro (descargamos y lo guardamos en el cluste)
wget https://raw.githubusercontent.com/uracilo/testdata/master/odisea.txt

5. Crear un directorio (creamos un directorio llamado input)
mkdir input


6. Crear un archivo tipo tar.gz (formato de compresion de archivos)

tar -czvf input/odisea.tar.gz odisea.txt

en este punto se creo el tar.gz y se guardo en la carpeta input

-c: Generar archivo -z: Comprimir con gzip. -v: Progreso del proceso. -f: Especificar nombre del archivo.

7. Revisar los tamaños de nuestros archivos
ls -flarts input
Output
```
total 292
  8 drwx------ 1 root root   4096 May 16 00:02 ..
  4 drwxr-xr-x 2 root root   4096 May 16 00:04 .
280 -rw-r--r-- 1 root root 284744 May 16 00:04 odisea.tar.gz

``` 

8. Crear y mover directorio input al DFS de HADOOP

hdfs dfs - (hace referencia al sistema de archivos del cluster)

hdfs dfs -mkdir -p test
hdfs dfs -put input

9. Revisar nuestro input directorio en HADOOP
hdfs dfs –ls  input

Output
```
Found 2 items
drwxr-xr-x   - root supergroup          0 2022-05-16 00:20 /user/root/input
drwxr-xr-x   - root supergroup          0 2022-05-16 00:20 /user/root/test


``` 

10. Leer las primeras lineas de nuestro archivo en HADOOP

-cat (investigar)
| (se utiliza para ejecutar multiples comando)
-zcar (descomprime el archivo)
-tail -n 20 cuenta las primeras 20 lineas 

hdfs dfs -cat  input/odisea.tar.gz | zcat | tail -n 20

11. Eliminar el archivo en HADOOP
hdfs dfs –rm –f /user/rawdata/example/odisea.tar.gz

Plus ejecutar un trabajo en HADOOP
hadoop jar $HADOOP_HOME/share/hadoop/mapreduce/sources/hadoop-mapreduce-examples-2.7.2-sources.jar org.apache.hadoop.examples.WordCount input output

Plus ver el resultado del trabajo en HADOOP
hdfs dfs -cat output/part-r-00000 (resultado del conteo de palabras)

salir del docker 

-abrir una nueva terminal y ejecutar:
sudo docker ps -a

Output
```
CONTAINER ID   IMAGE            COMMAND                  CREATED             STATUS                   PORTS                                                                                      NAMES
c62160e20827   uracilo/hadoop   "sh -c 'service ssh …"   About an hour ago   Up About an hour                                                                                                    hadoop-slave2
79e954de1b32   uracilo/hadoop   "sh -c 'service ssh …"   About an hour ago   Up About an hour                                                                                                    hadoop-slave1
68ec89918e76   uracilo/hadoop   "sh -c 'service ssh …"   About an hour ago   Up About an hour         0.0.0.0:8088->8088/tcp, :::8088->8088/tcp, 0.0.0.0:50070->50070/tcp, :::50070->50070/tcp   hadoop-master
56081a03218f   hello-world      "/hello"                 2 hours ago         Exited (0) 2 hours ago    
```

-detener un contenedor: 
sudo docker stop c62160e20827

sudo docker stop 79e954de1b32

sudo docker stop 68ec89918e76

-verificamos status de los contenedores: sudo docker ps -a

Output
```
CONTAINER ID   IMAGE            COMMAND                  CREATED             STATUS                            PORTS     NAMES
c62160e20827   uracilo/hadoop   "sh -c 'service ssh …"   About an hour ago   Exited (137) About a minute ago             hadoop-slave2
79e954de1b32   uracilo/hadoop   "sh -c 'service ssh …"   About an hour ago   Exited (137) About a minute ago             hadoop-slave1
68ec89918e76   uracilo/hadoop   "sh -c 'service ssh …"   About an hour ago   Exited (137) 24 seconds ago                 hadoop-master
56081a03218f   hello-world      "/hello"                 2 hours ago         Exited (0) 2 hours ago                      goofy_jepsen
```

-remover los contenedores:
sudo docker rm c62160e20827  

sudo docker rm 79e954de1b32

sudo docker rm 68ec89918e76

-verificamos el estatus: sudo docker ps -a

Output
```
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
```

-detener red: sudo docker network ls

Output
```
NETWORK ID     NAME      DRIVER    SCOPE
23ba8096c2fb   bridge    bridge    local (por defecto, siempre aparece)
90bddf54a168   hadoop    bridge    local
f5caa435456d   host      host      local (por defecto, siempre aparece)
e57cb08ba928   none      null      local (por defecto, siempre aparece)

```

-remover la red: sudo docker network rm 90bddf54a168

-verificar redes

Output
```
NETWORK ID     NAME      DRIVER    SCOPE
23ba8096c2fb   bridge    bridge    local (por defecto, siempre aparece)
f5caa435456d   host      host      local (por defecto, siempre aparece)
e57cb08ba928   none      null      local (por defecto, siempre aparece)

```


root@hadoop-master:~# sistema de archivos del nodo maestro

hdfs dfs (sistema de archivos remoto, se accede a los archivos de los demas servidores del cluster)
